In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from tabulate import tabulate

# Login

In [ ]:
user_sql = 'USER'
pass_sql = 'PWD'
server = 'SQNPRC001'
database = 'DBSV001'

string_conec = 'mssql+pyodbc://'+user_sql+':'+pass_sql+'@'+server+'/'+database+'?driver=SQL+Server+Native_Client+11.0'
engine = create_engine(string_conec,echo=False,pool_size=50,max_overflow=50)

# Search Table

In [ ]:
def search_table(substring):

    sql_tables = pd.read_sql('SELECT * FROM information_schema.tables', string_conec)

    sql_tables = sql_tables[sql_tables['TABLE_NAME'].str.contains(substring)]

    def concat_values(row):
        return f"{row['TABLE_SCHEMA']}.{row['TABLE_NAME']}"
    
    sql_tables['TABLE_NAME'] = sql_tables.apply(concat_values, axis = 1)

    sql_tables = sql_tables.reset_index(drop=True)

    sql_tables = sql_tables[['TABLE_NAME']]

    print(tabulate(sql_tables, headers='keys', tablefmt='pretty'))

In [ ]:
search_table('substring')

# Select Table

In [ ]:
table = 'dbo.tb_tablename'
schema, tablename = table.split('.')

# List Columns

In [ ]:
def list_columns(tablename):

    schema, table = tablename.split('.')

    types = pd.read_sql('exec sp_columns \''+table+'\', \''+schema++'\'', string_conec)
    types = types[['COLUMN_NAME', 'TYPE_NAME']]

    print(tabulate(sql_types, headers='heys', tablefmt='pretty'))

In [ ]:
create_script(table)

# Check Primary Keys

In [ ]:
def primary_key(tablename):

    tablename = 'dbo.tb_tablename'

    schema, table = tablename.split('.')

    pk = pd.read_sql(f'''
    
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
    WHERE OBJECTPROPERTY(OBJECT_ID(CONSTRAINT_SCHEMA + '.' + CONSTRAINT_NAME), 'IsPrimaryKey') = 1
    AND TABLE_SCHEMA = '{schema}'
    AND TABLE_NAME = '{table}'

    ''')

    print(tabulate(pk, headers='keys', tablefmt='pretty'))

In [ ]:
primary_key(table)

# Read query

In [ ]:
df = pd.read_sql('''

SELECT
    col1,
    col2

FROM table
WHERE date >= '2023-03-14'
ORDER BY id

''', string_conec)

# Delete

In [ ]:
engine.execute(f'DELETE FROM {table} WHERE ')

# Upload

In [ ]:
df.to_sql(name = table, con = engine, schema = schema, index = False, if_exists='append')